In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}




Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
cd drive/project_data/final_pascal_npy_datas

/content/drive/project_data/final_pascal_npy_datas


In [0]:
import keras
from keras.models import Sequential,load_model
from keras import layers, Model,Input
from keras.layers.normalization import BatchNormalization
from keras import optimizers
import numpy as np
import tensorflow as tf
from keras.optimizers import Adam
from keras import backend as K
from keras.layers.merge import concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing import image
from keras.engine.topology import Layer

Using TensorFlow backend.


# Losses

In [0]:
def custom_loss(y_true,y_pred):
    loss = tf.reduce_mean(y_true[0,:,0:,0:1]*tf.squared_difference(y_true[0,:,0:,1:5],y_pred[0,:,0:,1:5]))
    conf_loss = tf.reduce_mean(tf.squared_difference(y_true[0,:,0:,0:1],y_pred[0,:,0:,0:1]))
    return loss+conf_loss
# def custom_acc(y_true,y_pred):
#     loss = y_true[0,:,0:,0:1]*tf.squared_difference(y_true[0,:,0:,1:5],y_pred[0,:,0:,1:5])
#     conf_loss = tf.squared_difference(y_true[0,:,0:,0:1],y_pred[0,:,0:,0:1])
    
    
#

In [0]:
def c_loss_one(y_true,y_pred):
  loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
  return loss

In [0]:
X = np.load('X/2007_test.npy')
Y = np.load('Y/2007_test_labels.npy')
print(X.shape)
print(Y.shape)


(100, 300, 300, 3)
(100, 9, 9, 5)


# Custom Layer

In [0]:
class NLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(NLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=False)
        super(NLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return (x/255)

    def compute_output_shape(self, input_shape):
        return input_shape

# Model

In [0]:
img_input = Input(shape=(300, 300, 3))

# VGG-16
x = NLayer((300,300))(img_input)

x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv1',
                  trainable=False)(x)
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv2',
                  trainable=False)(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv1',
                  trainable=False)(x)
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv2',
                  trainable=False)(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv1',
                  trainable=False)(x)
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv2',
                  trainable=False)(x)
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv3',
                  trainable=False)(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv1',
                  trainable=False)(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv2',
                  trainable=False)(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv3',
                  trainable=False)(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv1',
                  trainable=False)(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv2',
                  trainable=False)(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv3',
                  trainable=False)(x)
# VGG over

x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

x = layers.Conv2D(100, (5, 5),
                  activation='relu',
                  padding='same',
                  name='block6_conv1',
                  trainable=False)(x)
x = layers.Conv2D(50, (5, 5),
                  activation='relu',
                  padding='same',
                  name='block6_conv2',
                  trainable=False)(x)
x = layers.Conv2D(5, (5, 5),
                  activation='relu',
                  padding='same',
                  name='block6_conv3',
                  trainable=False)(x)



"\n################################################\nx = layers.MaxPooling2D((3, 3), strides=(3, 3), name='tatti_pool')(x)\nx = layers.MaxPooling2D((2, 2), strides=(2, 2), name='tatti_pool2')(x)\nx = layers.Conv2D(1, (1),\n                  activation='relu',\n                  padding='same',\n                  name='tatti',\n                  trainable=False)(x)\n"

In [0]:
model = Model(img_input, x)

In [0]:
pred = model.predict(X[0:1])
pred.shape

(1, 9, 9, 5)

In [0]:
model.load_weights('classifier_weights.h5',by_name=True)

In [0]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss=custom_loss, optimizer="SGD",metrics=['accuracy'])

In [0]:
history0 = model.fit(X, Y, batch_size=25, epochs=100, verbose=1)

Epoch 1/100
100/100 [==============================] - 6s 55ms/step - loss: 0.0042 - acc: 0.0247
Epoch 2/100
100/100 [==============================] - 2s 21ms/step - loss: 0.0042 - acc: 0.0247
Epoch 3/100
100/100 [==============================] - 2s 20ms/step - loss: 0.0042 - acc: 0.0247
Epoch 4/100
100/100 [==============================] - 2s 20ms/step - loss: 0.0042 - acc: 0.0247
Epoch 5/100
100/100 [==============================] - 2s 20ms/step - loss: 0.0042 - acc: 0.0247
Epoch 6/100
100/100 [==============================] - 2s 20ms/step - loss: 0.0042 - acc: 0.0247
Epoch 7/100
100/100 [==============================] - 2s 21ms/step - loss: 0.0115 - acc: 0.0247
Epoch 8/100
100/100 [==============================] - 2s 20ms/step - loss: 0.0042 - acc: 0.0247
Epoch 9/100
100/100 [==============================] - 2s 21ms/step - loss: 0.0115 - acc: 0.0247
Epoch 10/100
100/100 [==============================] - 2s 20ms/step - loss: 0.0042 - acc: 0.0247
Epoch 11/100
100/100 [=======

In [0]:
out = model.predict(X[0:1])

In [0]:
out.shape

(1, 9, 9, 5)

In [0]:
out = np.reshape(out,(81,5))
act = np.reshape(Y[0], (81,5))

In [0]:
print(out.shape)
print(act.shape)

(81, 5)
(81, 5)


In [0]:
for i in range(81):
  if act[i,0] == 1:
    print("################")
    print("i = ", i)
    print(act[i])

################
i =  32
[1.00000000e+00 1.14730878e-03 2.03666667e-03 1.38810198e-03
 8.73333333e-04]
################
i =  40
[1.         0.00169972 0.0017     0.00324835 0.00324   ]


In [0]:
for i in range(81):
  #if out[i,0] == 1:
    print("################")
    print("i = ", i)
    print(act[i])
    
    
"""
ACTUAL VALUES
----------------------------------------------------------------------
i =  32
1.14730878e-03    2.03666667e-03    1.38810198e-03    8.73333333e-04
----------------------------------------------------------------------
0.00169972        0.0017            0.00324835        0.00324
----------------------------------------------------------------------
PREEDECTED VALUES
----------------------------------------------------------------------
at i = 32
1.14730878e-03    2.03666667e-03    1.38810198e-03    8.73333333e-04
----------------------------------------------------------------------
ar i = 40
0.00169972        0.0017            0.00324835        0.00324 
----------------------------------------------------------------------
"""

################
i =  0
[0. 0. 0. 0. 0.]
################
i =  1
[0. 0. 0. 0. 0.]
################
i =  2
[0. 0. 0. 0. 0.]
################
i =  3
[0. 0. 0. 0. 0.]
################
i =  4
[0. 0. 0. 0. 0.]
################
i =  5
[0. 0. 0. 0. 0.]
################
i =  6
[0. 0. 0. 0. 0.]
################
i =  7
[0. 0. 0. 0. 0.]
################
i =  8
[0. 0. 0. 0. 0.]
################
i =  9
[0. 0. 0. 0. 0.]
################
i =  10
[0. 0. 0. 0. 0.]
################
i =  11
[0. 0. 0. 0. 0.]
################
i =  12
[0. 0. 0. 0. 0.]
################
i =  13
[0. 0. 0. 0. 0.]
################
i =  14
[0. 0. 0. 0. 0.]
################
i =  15
[0. 0. 0. 0. 0.]
################
i =  16
[0. 0. 0. 0. 0.]
################
i =  17
[0. 0. 0. 0. 0.]
################
i =  18
[0. 0. 0. 0. 0.]
################
i =  19
[0. 0. 0. 0. 0.]
################
i =  20
[0. 0. 0. 0. 0.]
################
i =  21
[0. 0. 0. 0. 0.]
################
i =  22
[0. 0. 0. 0. 0.]
################
i =  23
[0. 0. 0. 0. 0.]
##

In [0]:
pred = model.predict(X_try[0:1])

In [0]:
pred.shape

(1, 9, 9, 5)

In [0]:
X_try = np.zeros((100,300,300,3))
Y_try = np.zeros((100,9,9,5))
for i in range(5):
  X_try[i] = X[0]/300
  Y_try[i] = Y[0]/300
  
for b in range(5):
  for i in range(9):
    for j in range(9):
      Y_try[b,i,j,0] *= 300

In [0]:
np.save('WrongX',X_try)
np.save('WrongY',Y_try)

In [0]:
ls

2007_test_labels.npy   2012_train_labels.npy      notebook/
2007_test_labels.txt   2012_train_labels.txt      weights_begin.h5
2007_test.txt          2012_train.txt             weights_history1.h5
2007_train_labels.npy  2012_val_labels.npy        WrongX.npy
2007_train_labels.txt  2012_val_labels.txt        WrongY.npy
2007_train.txt         2012_val.txt               X/
2007_val_labels.npy    classifier_weights.h5      Y/
2007_val_labels.txt    default_all_boxes_new.npy
2007_val.txt           drive/


In [0]:
X_try.shape

(300, 300, 3)

In [0]:
Y_try.shape

(9, 9, 5)

In [0]:
Y_try[1,3]

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [1.        , 0.34419263, 0.611     , 0.41643059, 0.262     ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

# C ustom LAyer

In [0]:
import keras
from keras.models import Sequential,load_model
from keras import layers, Model,Input
from keras.layers.normalization import BatchNormalization
from keras import optimizers
import numpy as np
import tensorflow as tf
from keras.optimizers import Adam
from keras import backend as K
from keras.layers.merge import concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing import image
from keras.engine.topology import Layer

Using TensorFlow backend.


In [0]:
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)



In [0]:
img_input = Input(shape=(300, 300, 3))

x = MyLayer((300,300,3))(img_input)



ValueError: ignored